In [3]:
using ParallelTemperingMonteCarlo
using BenchmarkTools,DelimitedFiles,StaticArrays,Random

By the end of this module, the ptmc_run function will be complete. First we aim to split the dependencies from MCRun, then I aim to test variable function definitions.

In [9]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details


mc_cycles = 400000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

Wanting to test the use of splatting and varargs.

In [5]:
import ParallelTemperingMonteCarlo.MCRun.equilibration_cycle!

In [4]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)

Total number of moves per MC cycle: 13



equilibration done
equilibration done


MC loop done.
[0.0003275710166429321, 0.00034038231628987886, 0.0003541875217686626, 0.0003771192914525007, 0.0003871271199260189, 0.00041166221823725176, 0.00043725182080965515, 0.0004481145198137168, 0.0004892415372905008, 0.000515104682943868, 0.0005564389189273487, 0.0005866300755424616, 0.0006168592165216777, 0.0006944227240404881, 0.0007638579947522499, 0.0008900401243935854, 0.0009648086561921595, 0.0010102312231438865, 0.001162060369344718, 0.0015909789581521694, 0.0017710695798688687, 0.0022553539037819675, 0.0027215931181169776, 0.003094108733468708, 0.003327247352742674, 0.0031633947149170456, 0.003019056592225612, 0.002654620244466205, 0.0023918416792751244, 0.0021811717549868217, 0.0020685078870861556, 0.00194995364088873]
done


Two versions of equilibration_cycle! exist, one requires (mc_states,move_strat,mc_params,results,pot,ensemble), the other, only (pot) if the splatting works appropriately, this needs to be correctly reordered

In [6]:
initaliser = (mc_states,move_strat,mc_params,results,pot,ensemble)

(MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], Spherical

In [10]:
mc_states,move_strat,ensemble,results,delta_en_hist,delta_r2,step,n_steps,a,v,r = equilibration_cycle!(initaliser...)

Total number of moves per MC cycle: 13



equilibration done


(MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-3.425394057763946, -6.913844944789298, -0.35278125241762637], [2.1143092750557897, -5.252348601798354, 0.30096211367684034], [-2.0556600856536815, -1.7045046621586468, 2.1007313974884574], [-6.532275378049498, -1.963261928726493, -1.2752960577644359], [0.6191727692731311, -7.978304729378645, -4.725982255625056], [-4.9962275667554925, -6.008593442097449, -6.094716959585111], [0.8073192634843194, 1.3496957931201865, -6.487017287374171], [2.514805217512681, 0.8607226346108662, -0.6524337490425524], [4.118155805538634, -3.5561702464557867, -5.993497354614803], [-3.0246511923546353, 2.919236909471291, -2.2131150028075024], [-5.039392630622439, -0.3940693040448866, -6.968467307623405], [-0.579549565432555, -3.524671503813088, -9.205940648119633], [-1.137479383791679, -2.5473599325966343, -3.403313246041449]], Spherical

# Process Results

The last portion of mc_run is to analyse results. It's probably best to combine these tasks into a function

In [8]:
function finalise_results(mc_states,mc_cycles,results)

    #Energy average
    n_sample = mc_params.mc_cycles / mc_params.mc_sample
    en_avg = [mc_states[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]
    en2_avg = [mc_states[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
    results.en_avg = en_avg
    #heat capacity
    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]
    #count stats 
    results.count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
    results.count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]

    println(results.heat_cap)

    return results

end

finalise_results (generic function with 1 method)

In [11]:
for i = 1:20000
    ptmc_cycle!(mc_states,results,move_strat,mc_params,pot,ensemble,n_steps,a,v,r,false,i,pwd(),delta_en_hist,delta_r2)
end


In [12]:
results  = finalise_results(mc_states,mc_cycles,results)

[0.0861509584366059, 0.08267626037496288, 0.07907834586969303, 0.075887641497636, 0.07275951230959232, 0.06954450003382141, 0.06663055252217479, 0.06385810388812337, 0.06108164445819545, 0.05826355609936735, 0.05583207496529264, 0.053347485991882944, 0.05092413449213425, 0.04853718880637328, 0.04639779537290644, 0.04408635356750463, 0.04196474768481652, 0.03977774305026977, 0.03781734885568901, 0.03598849631919687, 0.03390545236487022, 0.031720403694686636, 0.029876892607452753, 0.02804610916730171, 0.02455249296922606, 0.022823762658328955, 0.020692612251012776, 0.018720059479856103, 0.017152775990172467, 0.01539246838260396, 0.01474343098356939, 0.01320920805456004]


Output{Float64}(100, -0.005578923450316458, -0.0023105826344585615, Float64[], [-0.000267920784099373, -0.00026743185273712017, -0.0002664951440597083, -0.00026600500896262566, -0.0002653951665490028, -0.00026437473109604796, -0.00026367542674372667, -0.00026302081890072434, -0.00026210153060943155, -0.0002608238167072911  …  -0.0002380075694097019, -0.00023486685720095204, -0.00022363612772829117, -0.00021975021835944113, -0.00021307083438894676, -0.00020640151376796122, -0.00020142088227063617, -0.00019443393126043494, -0.00019393684797015974, -0.0001870246879086604], [0.0861509584366059, 0.08267626037496288, 0.07907834586969303, 0.075887641497636, 0.07275951230959232, 0.06954450003382141, 0.06663055252217479, 0.06385810388812337, 0.06108164445819545, 0.05826355609936735  …  0.029876892607452753, 0.02804610916730171, 0.02455249296922606, 0.022823762658328955, 0.020692612251012776, 0.018720059479856103, 0.017152775990172467, 0.01539246838260396, 0.01474343098356939, 0.0132092080545600

In [14]:
results.heat_cap

32-element Vector{Float64}:
 0.0861509584366059
 0.08267626037496288
 0.07907834586969303
 0.075887641497636
 0.07275951230959232
 0.06954450003382141
 0.06663055252217479
 0.06385810388812337
 0.06108164445819545
 0.05826355609936735
 ⋮
 0.02804610916730171
 0.02455249296922606
 0.022823762658328955
 0.020692612251012776
 0.018720059479856103
 0.017152775990172467
 0.01539246838260396
 0.01474343098356939
 0.01320920805456004